<h1>Simulating the action potential with random walk of ions</h1>

<h3>GroupXX: Us :p</h3>

<h2>Abstract</h2>

<h2>1 - Theory</h2>

**Teori**

<h2>2 - Excercises</h2>

**Excercises**

<h2>3 - Programming</h2>


In [ ]:
'''Task 3 - programming'''

import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import norm
from scipy.constants import Boltzmann as kb

N = 1000 # number of particles
steps = 100 # number of steps in the random walk simulation
particles = np.zeros(N) # vector containing the initial position (x=0) of all particles
h = 1 # length of a step taken by each particle per step/unit time delta t
T=1 # Temperature, arbitrarily defined, just a constant
beta_k = 0.1 # product of the two constants beta and k, relevant from task 5
beta = 1/(kb*T) # constant beta, kb is Boltzmanns constant and T is the temperature
k = beta_k/beta # exctract konstant k from beta*k

# This gives the probability threshold of which a particles travels right or left
# For task 3 (V = 0) Pm = 0.5
# Relevant from task 5 when v != 0
def P_min(x, V):
    rel_prob = np.exp(-beta*(V[steps+int(x)-h]-V[steps+int(x)+h]))
    Pp = 1/(1 + rel_prob)
    Pm = 1 - Pp
    return Pm


'''
Simulates random walk
For each step it generates a step vector, containing the step direction of all particles at that step
and updates the position of each particle accordingly
'''
def rand_walk(p_pos, V):
    for i in range(steps):
        steps_vec = np.random.rand(N)
        for x in p_pos:
            steps_vec[steps_vec >= P_min(x, V)] = 1
            steps_vec[steps_vec != 1] = -1
        p_pos += steps_vec
        params = norm.fit(p_pos)
    return p_pos, params

# Function for plotting the resulting distribution of particles
def plot_distribution(p_pos, V, V_name):
    p, params = rand_walk(p_pos, V)
    x_vec = np.arange(min(p), max(p))
    fitted_pdf = norm.pdf(x_vec, loc = params[0], scale = params[1])
    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    ax1.hist(p, bins="auto")
    ax2.plot(x_vec, fitted_pdf, color="r")
    plt.title(V_name)
    plt.show()

# zero potential
V0 = lambda x: 0
V0 = np.vectorize(V0)

V_vec = np.arange(-steps, steps+1)
V0_vec = V0(V_vec)

plot_distribution(particles, V0_vec, "V0")


In [ ]:
'''Task 5 - programming'''
# Potential exercise 5.1
V1 = lambda x: k*x


# Potential exercise 5.2
def V2(x):
    if -3*h < x and x < 3*h:
        return k
    else:
        return 0

# Potential exercise 5.3
def V3(x):
    if x < -3*h:
        return -k
    elif -3*h < x and x < 3*h:
        return k*(-1 + 2 * ((x + 3*h)/(6*h)))
    else:
        return k
    
V1 = np.vectorize(V1)
V2 = np.vectorize(V2)
V3 = np.vectorize(V3)

V1_vec = V1(V_vec)
V2_vec = V2(V_vec)
V3_vec = V3(V_vec)

beta_k_list = [0.1, 3, 5, 7, 10]
for elem in beta_k_list:
    beta_k = elem
    k = beta_k/beta
    plot_distribution(particles, V0_vec, "V0")
    particles = np.zeros(N)
    plot_distribution(particles, V1_vec, "V1")
    particles = np.zeros(N)
    plot_distribution(particles, V2_vec, "V2")
    particles = np.zeros(N)
    plot_distribution(particles, V3_vec, "V3")
    particles = np.zeros(N)

In [ ]:
'''Tasj 7'''
from scipy.constants import elementary_charge as elemc

L = 50
h = 1
V_0_Na = 1
V_0_K = 2
Cc = 0.07*elemc #M/V
Qc_out = 150 #mM
C_p = 0.1 #mM
x_membrane = np.arange(-h, h)

N_Na = 60 + 1450
N_K = 1400 + 50

Na_pos = np.array([L/4]*1450 + [-L/4]*60)
K_pos = np.array([L/4]*50 + [-L/4]*1400)

steps = 400

V_vec = np.arange(-steps, steps+1)

#potentials
def V_channel(x, V0):
    if -h <= x and x <= h:
        return V0
    else:
        return 0
    
def V_elec(Na_pos, K_pos):
    Qc_vec = np.zeros(len(V_vec))
    for i in range(len(V_vec)):
        Na_in = Na_pos[Na_pos < -h].size
        K_in = K_pos[K_pos < -h].size
        Qc_in = (Na_in + K_in)*C_p
        Qc = Qc_in - Qc_out
        Qc_vec[i] = Qc
    return Qc_vec/Cc

def V_diffv(V_vec ,V_elec, V_channel, Na_pos, K_pos, V0):
    V_c = V_channel(V_vec, V0)
    V_e = V_elec(Na_pos, K_pos)
    return V_c + V_e

V_channel = np.vectorize(V_channel)

V_vec_Na = V_diffv(V_vec, V_elec, V_channel, Na_pos, K_pos, V_0_Na)


def P_min_7(x, V):
    rel_prob = np.exp(-beta*(V[steps+int(x)-h]-V[steps+int(x)+h]))
    Pp = 1/(1 + rel_prob)
    Pm = 1 - Pp
    return Pm


def rand_walk_7(Na_pos, K_pos, V_diffv):
    for i in range(steps):
        steps_vec_Na = np.random.rand(Na_pos.size)
        steps_vec_K = np.random.rand(K_pos.size)
        V_vec_Na = V_diffv(V_vec, V_elec, V_channel, Na_pos, K_pos, V_0_Na)
        V_vec_K = V_diffv(V_vec, V_elec, V_channel, Na_pos, K_pos, V_0_K)
        for x in Na_pos:
            steps_vec_Na[steps_vec_Na >= P_min(x, V_vec_Na)] = 1
            steps_vec_Na[steps_vec_Na != 1] = -1
        for x in K_pos:
            steps_vec_K[steps_vec_K >= P_min(x, V_vec_K)] = 1
            steps_vec_K[steps_vec_K != 1] = -1
        Na_pos += steps_vec_Na
        K_pos += steps_vec_K
        params_Na = norm.fit(Na_pos)
        params_K = norm.fit(K_pos)
    return Na_pos, K_pos, params_Na, params_K

# Function for plotting the resulting distribution of particles
def plot_distribution_7(Na_pos, K_pos, V, V_name):
    pNa, params = rand_walk(Na_pos, K_pos, V)
    x_vec = np.arange(min(p), max(p))
    fitted_pdf = norm.pdf(x_vec, loc = params[0], scale = params[1])
    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    ax1.hist(p, bins="auto")
    ax2.plot(x_vec, fitted_pdf, color="r")
    plt.title(V_name)
    plt.show()







